In [24]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
import traceback
import inflect

# Define paths and project range
BASE_DIRECTORY = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/03_raw/rawdata_cluster1_4_style_C_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/03_raw/rawdata_cluster1_4_style_C_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/03_raw/scraping_cluster1_4_style_C_log.txt"
PROJECT_RANGE = range(488, 859)  # Example range for q_ids in Clusters 4

# Initialize DataFrames
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

# Initialize tracking variables
scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0

def clean_column_headers(headers):
    """Cleans column headers by normalizing, removing unwanted characters, and singularizing words."""
    p = inflect.engine()
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
            # Correct any mis‐spellings of “type of upgrade”
            header = re.sub(r'\btype of upgr\s*ade\b', 'type of upgrade', header)
            words = header.split()
            singular_words = [p.singular_noun(word) if p.singular_noun(word) else word for word in words]
            header = " ".join(singular_words)
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    return value

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
        "ADNU",
        "LDNU",
        "RNU"
    ]
    for phrase in phrases:
        if re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback if none found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]
    existing_desired = [col for col in desired_order if col in df.columns]
    remaining = [col for col in df.columns if col not in existing_desired]
    new_order = existing_desired + remaining
    df = df[new_order]
    return df

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}", file=log_file)
        return gps_coords.groups()
    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}", file=log_file)
        return project_coords.groups()
    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"
        longitude = lon if "E" in text.upper() else f"-{lon}"
        print(f"Found directional GPS coordinates: {(latitude, longitude)}", file=log_file)
        return (latitude, longitude)
    print("GPS coordinates not found.", file=log_file)
    return (None, None)

def adjust_rows_length(data_rows, headers):
    """
    Ensures each row in data_rows has exactly len(headers) columns.
    If a row is too short, it is padded with empty strings.
    If too long, it is truncated.
    """
    col_count = len(headers)
    for i in range(len(data_rows)):
        row = data_rows[i]
        if len(row) > col_count:
            data_rows[i] = row[:col_count]
        elif len(row) < col_count:
            data_rows[i].extend([""] * (col_count - len(row)))
            
def extract_table2(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 2 in the provided PDF.
    If no "Point of Interconnection" is found, it then searches for 
    "Description of Interconnection" using similar logic.
    """
    import re, traceback
    print(f"\nProcessing {pdf_path} for Table 2 extraction...", file=log_file)
    point_of_interconnection = None
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)
    # Pattern for fallback search:
    desc_pattern = re.compile(r"Description\s+of\s+Interconnection", re.IGNORECASE)
    
    table_settings_list = [
        {"horizontal_strategy": "text", "vertical_strategy": "lines", "snap_tolerance": 1},
        {"horizontal_strategy": "lines", "vertical_strategy": "lines", "snap_tolerance": 2},
    ]
    extraction_successful = False
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            table_pages = []
            # Identify pages that mention Table 2 or B.1
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*(?:2|B\.1)\b", text, re.IGNORECASE):
                    table_pages.append(i)
            if not table_pages:
                print("No Table 2 found in the PDF.", file=log_file)
                return None
            first_page = table_pages[0]
            last_page = table_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 1
            print(f"Table 2 starts on page {scrape_start + 1} and ends on page {scrape_end}", file=log_file)
            
            # ----- First search: "Point of Interconnection" -----
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for 'Point of Interconnection'...", file=log_file)
                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)
                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                            continue
                        print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                        for row_num, row in enumerate(tab, start=1):
                            print(f"Row {row_num}: {row}", file=log_file)
                        for row_index, row in enumerate(tab, start=1):
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    poi_col_index = cell_index
                                    adjacent_col_index = poi_col_index + 1
                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:
                                            point_of_interconnection = poi_value
                                            print(f"\nFound Point of Interconnection: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break
                                        else:
                                            print(f"\n'Point of Interconnection' label found but adjacent value is empty (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            poi_value_parts = []
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)
                                            print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.", file=log_file)
                                            for scan_row_index in range(start_scan, end_scan):
                                                if scan_row_index == current_row_idx:
                                                    continue
                                                scan_row = tab[scan_row_index]
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                                        print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                    elif poi_pattern.search(scan_cell):
                                                        print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                        continue
                                            if poi_value_parts:
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index})", file=log_file)
                                                extraction_successful = True
                                                break
                                            else:
                                                print(f"\nNo POI value found in the surrounding rows (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                    else:
                                        print(f"\nPoint of Interconnection label found but no adjacent column (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                            if extraction_successful:
                                break
                        if extraction_successful:
                            break
                    if extraction_successful:
                        break
                if extraction_successful:
                    break

            # ----- Second search: "Description of Interconnection" -----
            if not extraction_successful or not (point_of_interconnection and point_of_interconnection.strip()):
                print("\nNo valid 'Point of Interconnection' found. Trying 'Description of Interconnection'...", file=log_file)
                extraction_successful_desc = False
                for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                    page = pdf.pages[page_number]
                    print(f"\nScraping tables on page {page_number + 1} for 'Description of Interconnection'...", file=log_file)
                    for attempt, table_settings in enumerate(table_settings_list, start=1):
                        print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                        tables = page.find_tables(table_settings=table_settings)
                        print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)
                        for table_index, table in enumerate(tables, start=1):
                            tab = table.extract()
                            if not tab:
                                print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                                continue
                            print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                            for row_num, row in enumerate(tab, start=1):
                                print(f"Row {row_num}: {row}", file=log_file)
                            for row_index, row in enumerate(tab, start=1):
                                for cell_index, cell in enumerate(row, start=1):
                                    if cell and desc_pattern.search(cell):
                                        desc_col_index = cell_index
                                        adjacent_col_index = desc_col_index + 1
                                        if adjacent_col_index <= len(row):
                                            poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                            if poi_value:
                                                point_of_interconnection = poi_value
                                                print(f"\nFound Description of Interconnection: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                                extraction_successful_desc = True
                                                break
                                            else:
                                                print(f"\n'Description of Interconnection' label found but adjacent value is empty (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                                poi_value_parts = []
                                                current_row_idx = row_index - 1
                                                start_scan = max(0, current_row_idx - 2)
                                                end_scan = min(len(tab), current_row_idx + 3)
                                                print(f"Scanning rows {start_scan + 1} to {end_scan} for Description value parts.", file=log_file)
                                                for scan_row_index in range(start_scan, end_scan):
                                                    if scan_row_index == current_row_idx:
                                                        continue
                                                    scan_row = tab[scan_row_index]
                                                    if adjacent_col_index - 1 < len(scan_row):
                                                        scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                        if scan_cell and not desc_pattern.search(scan_cell):
                                                            poi_value_parts.append(scan_cell)
                                                            print(f"Found Description part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                        elif desc_pattern.search(scan_cell):
                                                            print(f"Encountered another Description label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                            continue
                                                if poi_value_parts:
                                                    point_of_interconnection = " ".join(poi_value_parts)
                                                    print(f"\nConcatenated Description of Interconnection: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index})", file=log_file)
                                                    extraction_successful_desc = True
                                                    break
                                                else:
                                                    print(f"\nNo Description value found in the surrounding rows (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                        else:
                                            print(f"\nDescription label found but no adjacent column (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                if extraction_successful_desc:
                                    break
                            if extraction_successful_desc:
                                break
                        if extraction_successful_desc:
                            break
                    if extraction_successful_desc:
                        break
                if extraction_successful_desc:
                    extraction_successful = True

    except Exception as e:
        print(f"Error processing Table 2 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None

    if not extraction_successful:
        if point_of_interconnection is not None and point_of_interconnection != "":
            print("Interconnection label found but no adjacent value.", file=log_file)
            return "Value Missing"
        else:
            print("Neither 'Point of Interconnection' nor 'Description of Interconnection' found in Table 2.", file=log_file)
            return None

    return point_of_interconnection


def fix_column_names(columns):
    """
    Renames duplicate and empty column names.
    Duplicate names are suffixed with _1, _2, etc.
    Empty or whitespace-only names are replaced with unnamed_1, unnamed_2, etc.
    """
    new_cols = []
    counts = {}
    unnamed_count = 1
    for col in columns:
        # Treat empty or whitespace-only names as unnamed.
        if not col or col.strip() == "":
            new_col = f"unnamed_{unnamed_count}"
            unnamed_count += 1
        else:
            new_col = col.strip()
        if new_col in counts:
            new_col_with_suffix = f"{new_col}_{counts[new_col]}"
            counts[new_col] += 1
            new_cols.append(new_col_with_suffix)
        else:
            counts[new_col] = 1
            new_cols.append(new_col)
    return new_cols

def post_process_columns(df, log_file):
    """
    Post-processes DataFrame column names:
      1. For any column named 'unnamed_#' (or empty), look at its first non-empty cell.
         If that cell is not a dollar amount (i.e. does not match /^\$\s*\d+(?:,\d{3})*(?:\.\d+)?$/)
         and it contains 2 or 3 words, then rename the column to that value (after cleaning).
         If a column already exists with that name, merge the data from the renamed column into the
         existing column and drop the renamed column.
      2. If a column is named "Needed For", then rename it to "description" (merging with an existing
         description column if necessary).
    """
    # Process unnamed columns.
    for col in list(df.columns):
        if col.lower().startswith("unnamed_") or col.strip() == "":
            # Find the first non-empty cell in this column.
            first_non_empty = None
            for val in df[col]:
                cell_val = ""
                if isinstance(val, str):
                    cell_val = val.strip()
                elif val is not None:
                    cell_val = str(val).strip()
                if cell_val:
                    first_non_empty = cell_val
                    break
            if first_non_empty:
                # Check if the value is a dollar amount.
                if not re.match(r"^\$\s*\d+(?:,\d{3})*(?:\.\d+)?$", first_non_empty):
                    words = first_non_empty.split()
                    if 2 <= len(words) <= 3:
                        # Clean the candidate name.
                        new_name = clean_column_headers([first_non_empty])[0]
                        log_file.write(f"Renaming column '{col}' to '{new_name}' based on first non-empty value '{first_non_empty}'.\n")
                        if new_name in df.columns and new_name != col:
                            # Merge the two columns: fill empty cells in existing new_name from the renamed col.
                            for idx in df.index:
                                existing_val = df.at[idx, new_name]
                                candidate_val = df.at[idx, col]
                                if (pd.isna(existing_val) or existing_val == "") and (not pd.isna(candidate_val) and candidate_val != ""):
                                    df.at[idx, new_name] = candidate_val
                            df.drop(columns=[col], inplace=True)
                        else:
                            df.rename(columns={col: new_name}, inplace=True)
    # Process "Needed For" column: rename or merge it into "description".
    if "Needed For" in df.columns:
        if "description" in df.columns:
            log_file.write("Merging 'Needed For' column into existing 'description' column.\n")
            for idx in df.index:
                desc_val = df.at[idx, "description"]
                needed_for_val = df.at[idx, "Needed For"]
                if (pd.isna(desc_val) or desc_val == "") and (not pd.isna(needed_for_val) and needed_for_val != ""):
                    df.at[idx, "description"] = needed_for_val
            df.drop(columns=["Needed For"], inplace=True)
        else:
            log_file.write("Renaming 'Needed For' column to 'description'.\n")
            df.rename(columns={"Needed For": "description"}, inplace=True)
    return df

def extract_table3(pdf_path, log_file, is_addendum=False):
    """
    Extracts Table 3 data   the provided PDF.
     
      2. Renaming of duplicate/empty columns (using fix_column_names) and then post-processing
         unnamed columns as described.
    """
    print(f"\nProcessing {pdf_path} for Table 10", file=log_file)
    extracted_tables = []
    specific_phrase = None
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify pages that contain either Table 3 patterns or Attachment 1/Attachment 2.
            table3_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*10[-.]([1-3])\b", text, re.IGNORECASE):
                    #re.search(r"Attachment\s*[12]", text, re.IGNORECASE)):
                    table3_pages.append(i)
            if not table3_pages:
                print("No Table 10  found in the PDF.", file=log_file)
                return pd.DataFrame()
            first_page = table3_pages[0]
            last_page = table3_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 1
            print(f"Candidate pages start on {scrape_start + 1} and end on {scrape_end}", file=log_file)
            # Process each page that might contain table data.
            for page_number in range(scrape_start, min(scrape_end, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1}...", file=log_file)
                # This variable keeps track of the bottom y-coordinate of the previous table on the page.
                previous_table_bottom = None
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })
                for table_index, table in enumerate(tables):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.", file=log_file)
                        continue
                    table_bbox = table.bbox  # (x0, top, x1, bottom)
                    # Define the title region for the table: above the table bounding box.
                    title_bbox = (0, 0, page.width, table_bbox[1])
                    title_text = page.within_bbox(title_bbox).extract_text() or ""
                    table_title = None
                    if title_text:
                        title_lines = title_text.split('\n')[::-1]
                        for line in title_lines:
                            line = line.strip()
                            match = re.search(r"(Modification\s+of\s+)?Table\s*10[-.]?\d*[:\-\s]*(.*)", line, re.IGNORECASE)
                            if match:
                                table_title = match.group(2).strip()
                                break
                        
                  
                    # Extract the specific phrase using the refined table title.
                    if table_title:
                        specific_phrase = extract_specific_phrase(table_title)
                        print(f"New table detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        headers = clean_column_headers(tab[0])
                        # Rename header 'type' to 'type of upgrade' if needed.
                        if "type" in headers and "type of upgrade" not in headers:
                            headers = [("type of upgrade" if h == "type" else h) for h in headers]
                        if "need for" in headers:
                            headers = [("description" if h == "need for" else h) for h in headers]  
                    
                        # Apply the duplicate/empty column fixing.
                        headers = fix_column_names(headers)
                        data_rows = tab[1:]
                        try:
                            df_new = pd.DataFrame(data_rows, columns=headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue

                        if "allocated" in df_new.columns:
                            df_new.drop(columns=["allocated"], inplace=True)
                            print(f"Dropped 'Max of' column in table on page {page_number + 1}, table {table_index}.", file=log_file)

                        if "cost rate x " in df_new.columns:
                            df_new.drop(columns=["cost rate x "], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file)

                        if "cost rate" in df_new.columns:
                            df_new.drop(columns=["cost rate"], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file) 

                        if "3339615 9" in df_new.columns:
                            df_new.drop(columns=["3339615 9"], inplace=True)
                            print(f"Dropped '3339615 9' column in table on page {page_number + 1}, table {table_index}.", file=log_file)     
                            
                        if "6 steady state reliability and posttransient voltage stability" in df_new.columns:
                            df_new.drop(columns=["6 steady state reliability and posttransient voltage stability"], inplace=True)
                            print(f"Dropped '6 steady state reliability and posttransient voltage stability' column in table on page {page_number + 1}, table {table_index}.", file=log_file)  



                        if "escalated" in df_new.columns:
                            df_new.drop(columns=["escalated"], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file)    


                        # Also, if the DataFrame has a column named "type" (and not already "type of upgrade"), rename it.
                        if 'type' in df_new.columns and 'type of upgrade' not in df_new.columns:
                            df_new.rename(columns={'type': 'type of upgrade'}, inplace=True)
                        # Special handling for ADNU tables if needed.
                        if re.search(r"Area\s*Delivery\s*Upgrades", specific_phrase, re.IGNORECASE):
                            print("Detected 'Area Delivery Network Upgrade' table (new).", file=log_file)
                            if "adnu" in df_new.columns:
                                if "type of upgrade" not in df_new.columns:
                                    adnu_values = df_new["adnu"].dropna().astype(str).tolist()
                                    grouped_adnu = " ".join(adnu_values)
                                    other_columns = df_new.drop(columns=["adnu"]).iloc[0].to_dict()
                                    df_grouped = pd.DataFrame({
                                        "upgrade": [grouped_adnu],
                                        "type of upgrade": [specific_phrase]
                                    })
                                    for col, value in other_columns.items():
                                        df_grouped[col] = value
                                    print("Grouped all 'adnu' rows into a single 'upgrade' row for new ADNU table.", file=log_file)
                                    df_new = df_grouped
                                else:
                                    if "upgrade" in df_new.columns:
                                        df_new.drop(columns=['adnu'], inplace=True)
                                        print("Dropped 'adnu' column to avoid duplicate 'upgrade'.", file=log_file)
                                    else:
                                        df_new.rename(columns={'adnu': 'upgrade'}, inplace=True)
                                        print("Renamed 'adnu' to 'upgrade' in new ADNU table.", file=log_file)
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new ADNU table.", file=log_file)
                            else:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                                    print("Replaced None in 'type of upgrade' for new ADNU table.", file=log_file)
                        else:
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new non-ADNU table.", file=log_file)
                            else:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print("Replacing None in 'type of upgrade' for new non-ADNU table.", file=log_file)
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                        # Fix duplicate and unnamed columns in the new table.
                        df_new.columns = fix_column_names(df_new.columns.tolist())
                        # Now apply the post-processing of column names:
                        df_new = post_process_columns(df_new, log_file)
                        extracted_tables.append(df_new)
                    else:
                        # Continuation table branch.
                        if specific_phrase is None:
                            print(f"No previous table title found for continuation on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue
                        print(f"Continuation table detected on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        data_rows = tab
                        # Use the number of columns from the last extracted table as expected.
                        expected_columns = len(extracted_tables[-1].columns) if extracted_tables else None
                        if expected_columns is None:
                            print(f"No existing table to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue
                        expected_headers = extracted_tables[-1].columns.tolist()
                        header_keywords = ["type of upgrade", "adnu"]
                        first_continuation_row = data_rows[0] if data_rows else []
                        is_header_row = any(
                            re.search(rf"\b{kw}\b", str(cell), re.IGNORECASE) for kw in header_keywords for cell in first_continuation_row
                        )
                        if is_header_row:
                            print(f"Detected header row in continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            data_rows = data_rows[1:]
                        # Ensure every row has the same length as expected_headers.
                        adjust_rows_length(data_rows, expected_headers)
                        try:
                            df_continuation = pd.DataFrame(data_rows, columns=expected_headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue
                        # Rename column 'type' if needed.
                        if 'type' in df_continuation.columns and 'type of upgrade' not in df_continuation.columns:
                            df_continuation.rename(columns={'type': 'type of upgrade'}, inplace=True)
                        if "need for" in df_continuation.columns:
                            df_continuation.rename(columns={"need for": "description"}, inplace=True)
                        if re.search(r"Area\s*Delivery\s*Upgrades", specific_phrase, re.IGNORECASE):
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing None in 'type of upgrade' for continuation ADNU table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'type of upgrade' column added with value '{specific_phrase}' for continuation ADNU table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        else:
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing None in 'type of upgrade' for continuation table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'type of upgrade' column added with value '{specific_phrase}' for continuation table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        # Fix duplicate and unnamed columns in the continuation table.
                        df_continuation.columns = fix_column_names(df_continuation.columns.tolist())
                        # Post-process the columns in the continuation table.
                        df_continuation = post_process_columns(df_continuation, log_file)
                        # Concatenate the continuation table with the previous extracted table.
                        extracted_tables[-1] = pd.concat([extracted_tables[-1], df_continuation], ignore_index=True, sort=False)
                    # Update the previous_table_bottom for the page using the current table's bbox.
                    previous_table_bottom = table_bbox[3]
    except Exception as e:
        print(f"Error processing Table 10 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()
    if extracted_tables:
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())
        standardized_tables = []
        for df in extracted_tables:
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)
        print("\nConcatenating all extracted Table 10/Attachment data...", file=log_file)
        try:
            table3_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
            print(f"Successfully concatenated {len(standardized_tables)} tables.", file=log_file)
        except Exception as e:
            print(f"Error concatenating tables: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            table3_data = pd.DataFrame()
    else:
        print("No Table 10/Attachment data extracted.", file=log_file)
        table3_data = pd.DataFrame()
    return table3_data


def extract_table3_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """Extracts Table 10 data and merges with base data."""
    base_data = extract_base_data(pdf_path, project_id, log_file)
    table3_data = extract_table3(pdf_path, log_file, is_addendum)
    if table3_data.empty:
        return base_data
    else:
        overlapping_columns = base_data.columns.intersection(table3_data.columns).difference(['point_of_interconnection'])
        table3_data = table3_data.drop(columns=overlapping_columns, errors='ignore')
        base_data_repeated = pd.concat([base_data] * len(table3_data), ignore_index=True)
        try:

                        # Concatenate base data with Table 8 data along columns
            merged_df = pd.concat([base_data_repeated, table3_data], axis=1, sort=False)
           # if "q_id" in merged_df.columns and "type of upgrade" in merged_df.columns and "upgrade" in merged_df.columns:
           #     merged_df["is_duplicate"] = merged_df.duplicated(subset=["q_id", "type of upgrade", "upgrade"], keep="first")
            #    merged_df = merged_df[merged_df["is_duplicate"] == False].drop(columns=["is_duplicate"])
            #    print(f"Removed duplicate rows based on 'q_id', 'type of upgrade', and 'upgrade'.", file=log_file)


            if "q_id" in merged_df.columns and "type of upgrade" in merged_df.columns and "upgrade" in merged_df.columns:
                # Identify rows where 'type of upgrade' and 'upgrade' are not empty
                non_empty_rows = merged_df[
                    merged_df["type of upgrade"].notna() & merged_df["upgrade"].notna() &
                    (merged_df["type of upgrade"].str.strip() != "") & (merged_df["upgrade"].str.strip() != "")
                ]

                # Group by q_id, type of upgrade, and upgrade, keeping the first occurrence
                grouped_df = non_empty_rows.groupby(["q_id", "type of upgrade", "upgrade"], as_index=False).first()

                # Get the original order of the rows in merged_df before filtering
                merged_df["original_index"] = merged_df.index

                # Combine unique grouped rows with originally empty rows
                final_df = pd.concat([
                    grouped_df,
                    merged_df[merged_df["type of upgrade"].isna() | (merged_df["type of upgrade"].str.strip() == "") |
                            merged_df["upgrade"].isna() | (merged_df["upgrade"].str.strip() == "")]
                ], ignore_index=True, sort=False)

                # Restore the original order of the rows based on the saved index
                final_df.sort_values(by="original_index", inplace=True)
                final_df.drop(columns=["original_index"], inplace=True)
                merged_df = final_df

                print(f"Removed duplicate rows based on 'q_id', 'type of upgrade', and 'upgrade', excluding empty rows while preserving order.", file=log_file)

            merged_df = pd.concat([base_data_repeated, table3_data], axis=1, sort=False)
            if 'point_of_interconnection' not in merged_df.columns:
                merged_df['point_of_interconnection'] = base_data['point_of_interconnection'].iloc[0]
                print(f"Added 'point_of_interconnection' to merged data for {pdf_path}.", file=log_file)
            print(f"Merged base data with Table 3 data for {pdf_path}.", file=log_file)
            return merged_df
        except Exception as e:
            print(f"Error merging base data with Table 3 data for {pdf_path}: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return base_data

def check_has_table3(pdf_path):
    """Checks if the PDF contains Table 3 or Attachment 1/Attachment 2."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                if re.search(r"(Modification\s+of\s+)?Table\s*10[-.]?\d*", text, re.IGNORECASE):
                    #re.search(r"Attachment\s*[12]", text, re.IGNORECASE))
                    return True
    except Exception as e:
        return False
    return False

def is_addendum(pdf_path):
    """Checks if the PDF is an addendum by searching 'Addendum' on the first page."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            first_page = pdf.pages[0]
            text = first_page.extract_text() or ""
            return "Addendum" in text
    except Exception as e:
        return False

def extract_base_data(pdf_path, project_id, log_file):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...", file=log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
        text = clean_string_cell(text)
        #queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = str(project_id)
        print(f"Extracted Queue ID: {queue_id}", file=log_file)
        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1) if cluster_number else None
        print(f"Extracted Cluster Number: {cluster_number}", file=log_file)
        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}", file=log_file)
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}", file=log_file)
        point_of_interconnection = extract_table2(pdf_path, log_file)
        latitude, longitude = search_gps_coordinates(text, log_file)
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }
        print("Base data extracted:", file=log_file)
        print(base_data, file=log_file)
        return pd.DataFrame(base_data)
    except Exception as e:
        print(f"Error extracting base data from {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return
    df = df.map(clean_string_cell)
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")
    #if 'q_id' in df.columns:
    #   df['q_id'] = pd.to_numeric(df['q_id'], errors='coerce')
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

import os
import re
# (Other imports remain the same)

def process_pdfs_in_folder():
    """
    Processes all PDFs in the directories within BASE_DIRECTORY whose numeric prefix is in PROJECT_RANGE.
    This allows folders like '641' and '641AA' (if 641 is in the PROJECT_RANGE) to be processed,
    and uses the full folder name (e.g. '641AA') as the project id (q_id).
    """
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped
    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)
 


    with open(LOG_FILE_PATH, 'w') as log_file:
        # List all subdirectories in BASE_DIRECTORY that have a numeric prefix.
        folders = [
            folder for folder in os.listdir(BASE_DIRECTORY)
            if os.path.isdir(os.path.join(BASE_DIRECTORY, folder)) and re.match(r'^(\d+)', folder)
        ]
    


        def sort_key(folder):
            match = re.match(r'^(\d+)', folder)
            if match:
                numeric = int(match.group(1))
                return (numeric, folder)
            return (float('inf'), folder)

        # Sort the folders in ascending order.
        sorted_folders = sorted(folders, key=sort_key)

        # Process each folder in sorted order.
        for folder in sorted_folders:
            folder_path = os.path.join(BASE_DIRECTORY, folder)
            match = re.match(r'^(\d+)', folder)
            if not match:
                continue  # Skip if there is no numeric prefix.
            numeric_part = int(match.group(1))
            # Process the folder only if its numeric part is in the desired range.
            if numeric_part not in PROJECT_RANGE:
                continue

            # Use the full folder name as the project identifier (q_id).
            project_id = folder  # e.g., "641AA" or "641"
            project_path = os.path.join(folder_path, "02_phase_1_study")
            if not os.path.exists(project_path):
                missing_projects.add(project_id)
                print(f"Project path does not exist: {project_path}", file=log_file)
                continue

            project_scraped = False
            base_data_extracted = False
            base_data = pd.DataFrame()
            for pdf_name in os.listdir(project_path):
                if pdf_name.endswith(".pdf"):
                    pdf_path = os.path.join(project_path, pdf_name)
                    total_pdfs_accessed += 1
                    is_add = is_addendum(pdf_path)
                    if is_add:
                        addendum_pdfs.append(pdf_name)
                        print(f"Accessing Addendum PDF: {pdf_name} from Project {project_id}", file=log_file)
                    else:
                        original_pdfs.append(pdf_name)
                        print(f"Accessing Original PDF: {pdf_name} from Project {project_id}", file=log_file)
                    try:
                        has_table3 = check_has_table3(pdf_path)
                        if not has_table3:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3 or Attachment data)", file=log_file)
                            print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3 or Attachment data)")
                            total_pdfs_skipped += 1
                            continue
                        if not is_add and not base_data_extracted:
                            base_data = extract_base_data(pdf_path, project_id, log_file)
                            base_data_extracted = True
                            print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)
                        if is_add and base_data_extracted:
                            table3_data = extract_table3(pdf_path, log_file, is_addendum=is_add)
                            if not table3_data.empty:
                                merged_df = pd.concat([base_data] * len(table3_data), ignore_index=True)
                                merged_df = pd.concat([merged_df, table3_data], axis=1, sort=False)
                                core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)")
                                total_pdfs_skipped += 1
                        else:
                            df = extract_table3_and_replace_none(pdf_path, project_id, log_file, is_addendum=is_add)
                            if not df.empty:
                                if is_add:
                                    core_addendums = pd.concat([core_addendums, df], ignore_index=True)
                                else:
                                    core_originals = pd.concat([core_originals, df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                                print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)")
                                total_pdfs_skipped += 1
                    except Exception as e:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                        print(traceback.format_exc(), file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                        total_pdfs_skipped += 1
            if not project_scraped and os.path.exists(project_path):
                skipped_projects.add(project_id)
    # Save results and print summary as before.
    save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
    save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")
    total_projects_processed = len(scraped_projects) + len(skipped_projects)
    print("\n=== Scraping Summary ===")
    print(f"Total Projects Processed: {total_projects_processed}")
    print(f"Total Projects Scraped: {len(scraped_projects)}")
    print(f"Total Projects Skipped: {len(skipped_projects)}")
    print(f"Total Projects Missing: {len(missing_projects)}")
    print(f"Total PDFs Accessed: {total_pdfs_accessed}")
    print(f"Total PDFs Scraped: {total_pdfs_scraped}")
    print(f"Total PDFs Skipped: {total_pdfs_skipped}")
    print("\nList of Scraped Projects:")
    print(sorted(scraped_projects))
    print("\nList of Skipped Projects:")
    print(sorted(skipped_projects))
    print("\nList of Missing Projects:")
    print(sorted(missing_projects))
    print("\nList of Scraped PDFs:")
    print(scraped_pdfs)
    print("\nList of Skipped PDFs:")
    print(skipped_pdfs)
    print("\nList of Addendum PDFs:")
    print(addendum_pdfs)
    print("\nList of Original PDFs:")
    print(original_pdfs)
    print("\nNumber of Original PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in original_pdfs]))
    print("Number of Addendum PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs]))


def main():
    """Main function to execute the PDF scraping process."""
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


Skipped PDF: Appendix A - C493_06-25-2010_Final.pdf from Project 493 (No Table 3 or Attachment data)
Scraped PDF: Q495_ Cluster 1_Report_Appendix A_final.pdf from Project 495
Skipped PDF: Appendix A - C510_06-25-2010_Final.pdf from Project 510 (No Table 3 or Attachment data)
Skipped PDF: QC4PI-SCE-Northern-Appendix A-DS-Q521 Columbia 1.pdf from Project 521 (No Table 3 or Attachment data)
Skipped PDF: QC4PI-SCE-Northern-Appendix A-DS-Q522 Columbia 2.pdf from Project 522 (No Table 3 or Attachment data)
Scraped PDF: 10AS666541-Appendix_A__Q557_Cluster2_Phase_I_study_report_rev1.pdf from Project 557
Scraped PDF: Appendix A - Q557 Cluster2 Phase I study report.pdf from Project 557
Scraped PDF: 10AS665861-Appendix_A__Q559_Cluster2_Phase_I_study_report_rev1.pdf from Project 559
Scraped PDF: Appendix A - Q559 Cluster2 Phase I study report.pdf from Project 559
Scraped PDF: Appendix A - Q560 Cluster2 Phase I Study Report.pdf from Project 560
Scraped PDF: 10AS665827-Appendix_A__Q560_Cluster2_Phas

In [19]:
import os
import re
import pandas as pd

# =====================================================
# User Settings and File Paths (update these as needed)
# =====================================================

# Base folder containing all project folders
base_folder = r'/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data'

# Numeric range to consider (based on the numeric prefix of folder names)
range_start = 488   # (Update these values if needed)
range_end   = 859

# Path to your CSV file (which must have columns "q_id" and "cluster_number")
phase_status_csv = r'/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/phase_status.csv'

# =====================================================
# Read CSV and Build Cluster Mapping
# =====================================================

# Read the CSV ensuring that the q_id is treated as a string
phase_status = pd.read_csv(phase_status_csv, dtype={'q_id': str})
# Create a mapping from q_id to cluster_number
cluster_mapping = dict(zip(phase_status['q_id'], phase_status['cluster_number']))

# =====================================================
# Initialize Data Structures for Results
# =====================================================

empty_phase1_count = 0          # Count of projects with an empty "02_phase_1_study"
empty_phase1_folders = []       # List of folder paths with an empty "02_phase_1_study" subfolder

# Dictionaries to group projects by cluster:
# 1. Projects missing Phase 1 (i.e. the folder is missing or exists but is empty)
missing_phase1_by_cluster = {}  

# 2. Projects that have an empty Phase 1 folder but have non-empty "03_phase_2_study" or "05_reassesment"
empty_phase1_with_following_by_cluster = {}  

# List to hold all project folder names (for final sorted list)
project_folders = []

# =====================================================
# Gather and Sort Candidate Folders
# =====================================================

# We use a list of tuples (folder_name, numeric_prefix) so we can sort by the numeric value
folder_candidates = []
for item in os.listdir(base_folder):
    item_path = os.path.join(base_folder, item)
    if os.path.isdir(item_path):
        # Use regex to extract the numeric prefix (works for names like "641" or "641AA")
        m = re.match(r'^(\d+)', item)
        if m:
            numeric_part = int(m.group(1))
            if range_start <= numeric_part <= range_end:
                folder_candidates.append((item, numeric_part))

# Sort candidates in ascending order first by the numeric prefix, then alphabetically by the full folder name
folder_candidates.sort(key=lambda x: (x[1], x[0]))

# =====================================================
# Process Each Candidate Folder
# =====================================================

for folder_name, num in folder_candidates:
    # Add folder name to final list (for reporting later)
    project_folders.append(folder_name)
    
    folder_path = os.path.join(base_folder, folder_name)
    # Build subfolder paths
    phase1_path = os.path.join(folder_path, "02_phase_1_study")
    phase2_path = os.path.join(folder_path, "03_phase_2_study")
    reassessment_path = os.path.join(folder_path, "05_reassesment")
    
    # Determine the cluster for this project using the CSV mapping; default to "Unknown" if not found.
    cluster = cluster_mapping.get(folder_name, "Unknown")
    
    # Check for "missing" Phase 1:
    # Here, we define "missing" as: either the Phase 1 folder does not exist OR it exists but is empty.
    phase1_missing = False
    if not os.path.isdir(phase1_path):
        phase1_missing = True
    else:
        # The folder exists; check if it is empty.
        if not os.listdir(phase1_path):
            phase1_missing = True
            empty_phase1_count += 1
            empty_phase1_folders.append(folder_path)
    
    if phase1_missing:
        missing_phase1_by_cluster.setdefault(cluster, []).append(folder_name)
    
        # Additionally, if the folder exists (but is empty) and at least one of the follow-up folders is non-empty,
        # then add it to the separate grouping.
        if os.path.isdir(phase1_path):  # only check follow-ups if the folder exists (even though it's empty)
            phase2_non_empty = os.path.isdir(phase2_path) and bool(os.listdir(phase2_path))
            reassessment_non_empty = os.path.isdir(reassessment_path) and bool(os.listdir(reassessment_path))
            if phase2_non_empty or reassessment_non_empty:
                empty_phase1_with_following_by_cluster.setdefault(cluster, []).append(folder_name)

# =====================================================
# Output the Results
# =====================================================

print(f"\nTotal projects (from {range_start} to {range_end}) with a missing or empty '02_phase_1_study' folder: {len(sum(missing_phase1_by_cluster.values(), []))}")

if empty_phase1_folders:
    print("\nFolders with an empty '02_phase_1_study' subfolder (full paths):")
    for folder in empty_phase1_folders:
        print(folder)

print("\n=== Projects Missing '02_phase_1_study' (Missing or Empty) Grouped by Cluster ===")
if missing_phase1_by_cluster:
    for clust, folders in missing_phase1_by_cluster.items():
        print(f"\nCluster {clust}:")
        for folder in folders:
            print(f"  {folder}")
else:
    print("No projects found missing the '02_phase_1_study' folder.")

print("\n=== Projects with Empty '02_phase_1_study' but with Non-Empty '03_phase_2_study' or '05_reassesment' (Grouped by Cluster) ===")
if empty_phase1_with_following_by_cluster:
    for clust, folders in empty_phase1_with_following_by_cluster.items():
        print(f"\nCluster {clust}:")
        for folder in folders:
            print(f"  {folder}")
else:
    print("No projects found with empty '02_phase_1_study' but non-empty subsequent phases.")

print("\n=== All Project Folder Names (Ascending Order) ===")
# Build a list of just the folder names
project_folders = [folder for folder, num in folder_candidates]

# Print the list in a format you can copy and paste
# Print the list with 10 items per line
print("Project folder names (ascending order):")
line_length = 10
for i in range(0, len(project_folders), line_length):
    line_items = project_folders[i:i+line_length]
    # Join each item wrapped in quotes and separated by a comma and space
    line = ", ".join(f"'{item}'" for item in line_items)
    print(line)

# =====================================================
# Print Total Counts by Cluster
# =====================================================

# Get all clusters that appear in either dictionary.
all_clusters = set(missing_phase1_by_cluster.keys()).union(set(empty_phase1_with_following_by_cluster.keys()))

print("\n=== Total Count by Cluster ===")
for clust in sorted(all_clusters, key=lambda x: str(x)):
    count_missing = len(missing_phase1_by_cluster.get(clust, []))
    count_following = len(empty_phase1_with_following_by_cluster.get(clust, []))
    print(f"Cluster {clust}:")
    print(f"  Missing or Empty '02_phase_1_study': {count_missing}")
    print(f"  Empty '02_phase_1_study' with Follow-up Data: {count_following}")



Total projects (from 488 to 859) with a missing or empty '02_phase_1_study' folder: 94

Folders with an empty '02_phase_1_study' subfolder (full paths):
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/488
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/490
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/494
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/502
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/503
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/509
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/512
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/555
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/579
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/585
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/640
/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/642
/Users/vk365/Dropb

In [ ]:
#skipped by both styles but not present in the missing list:
 [521, a combined report for phase 1 and phase 2 present under phase 2
  522, 
  
  541, missing
  552, missing
  558, missing
  667, was scraped
    737, no table
    795] group report 


In [20]:
# Define the lists for Style C
styleC_scraped = ['495', '557', '559', '560', '568', '569', '576', '577', '581', '586', '588', '589', '593', '602', '606', '607', '632C', '643AB', '643AC', '643AE', '643AJ', '643D', '643E', '643F', '643G', '643J', '643O', '643R', '643S', '643X', '644', '645A', '650AC', '653B', '653E', '653EA', '653F', '654', '670', '671', '678', '679', '680', '687', '688', '695', '702', '705', '707', '708', '709', '714', '720', '723', '725', '738', '740', '744', '746', '751', '752', '762', '765', '768', '769', '775', '778', '779', '796', '797', '798', '799', '800', '805', '806', '809', '814', '815', '816', '820', '823', '824', '825', '829', '831', '855', '856']

styleC_skipped = ['488', '490', '493', '494', '502', '503', '506', '509', '510', '512', '513', '521', '522', '522C', '541', '552', '555', '558', '561', '565', '574', '579', '583', '585', '590', '608', '628', '632AA', '640', '642', '643', '643AA', '643AF', '643AH', '643AI', '643AK', '643AM', '643AP', '643AS', '643I', '643T', '643W', '643Z', '644A', '647', '649', '649A', '649B', '649C', '650A', '650AA', '651', '651A', '653', '653A', '653D', '653EB', '653ED', '653H', '658', '659', '660', '661', '662', '663', '664', '667', '668', '669', '674', '676', '681', '683', '684', '685', '686', '692', '696', '697', '698', '700', '703', '704', '706', '712', '716', '717', '729', '730', '732', '736', '737', '739', '741', '756', '764', '766', '767', '770', '771', '774', '781', '782', '783', '784', '785', '786', '788', '789', '790', '791', '792', '793', '794', '795', '801', '804', '807', '812', '813', '834', '836', '837', '838', '839', '840', '841', '842', '843', '845', '846', '847', '848', '849', '850', '851', '852', '853', '854', '857', '858']

# Define the lists for Style R
styleR_scraped = ['493', '510', '561', '565', '574', '583', '590', '608', '628', '643AM', '643AP', '643T', '644A', '649B', '649C', '650AA', '651A', '653ED', '653H', '658', '659', '660', '661', '662', '667', '781', '789', '794', '837', '838']

styleR_skipped = ['488', '490', '494', '495', '502', '503', '506', '509', '512', '513', '521', '522', '522C', '541', '552', '555', '557', '558', '559', '560', '568', '569', '576', '577', '579', '581', '585', '586', '588', '589', '593', '602', '606', '607', '632AA', '632C', '640', '642', '643', '643AA', '643AB', '643AC', '643AE', '643AF', '643AH', '643AI', '643AJ', '643AK', '643AS', '643D', '643E', '643F', '643G', '643I', '643J', '643O', '643R', '643S', '643W', '643X', '643Z', '644', '645A', '647', '649', '649A', '650A', '650AC', '651', '653', '653A', '653B', '653D', '653E', '653EA', '653EB', '653F', '654', '663', '664', '668', '669', '670', '671', '674', '676', '678', '679', '680', '681', '683', '684', '685', '686', '687', '688', '692', '695', '696', '697', '698', '700', '702', '703', '704', '705', '706', '707', '708', '709', '712', '714', '716', '717', '720', '723', '725', '729', '730', '732', '736', '737', '738', '739', '740', '741', '744', '746', '751', '752', '756', '762', '764', '765', '766', '767', '768', '769', '770', '771', '774', '775', '778', '779', '782', '783', '784', '785', '786', '788', '790', '791', '792', '793', '795', '796', '797', '798', '799', '800', '801', '804', '805', '806', '807', '809', '812', '813', '814', '815', '816', '820', '823', '824', '825', '829', '831', '834', '836', '839', '840', '841', '842', '843', '845', '846', '847', '848', '849', '850', '851', '852', '853', '854', '855', '856', '857', '858']



empty= ['488'
'490', '493', '494', '495', '502', '503', '506', '509', '510', '512'
'513', '521', '522', '522C', '541', '552', '555', '557', '558', '559'
'560', '561', '565', '568', '569', '574', '576', '577', '579', '581'
'583', '585', '586', '588', '589', '590', '593', '602', '606', '607'
'608', '628', '632AA', '632C', '640', '642', '643', '643AA', '643AB', '643AC'
'643AE', '643AF', '643AH', '643AI', '643AJ', '643AK', '643AM', '643AP', '643AS', '643D'
'643E', '643F', '643G', '643I', '643J', '643O', '643R', '643S', '643T', '643W'
'643X', '643Z', '644', '644A', '645A', '647', '649', '649A', '649B', '649C'
'650A', '650AA', '650AC', '651', '651A', '653', '653A', '653B', '653D', '653E'
'653EA', '653EB', '653ED', '653F', '653H', '654', '658', '659', '660', '661'
'662', '663', '664', '667', '668', '669', '670', '671', '674', '676'
'678', '679', '680', '681', '683', '684', '685', '686', '687', '688'
'692', '695', '696', '697', '698', '700', '702', '703', '704', '705'
'706', '707', '708', '709', '712', '714', '716', '717', '720', '723'
'725', '729', '730', '732', '736', '737', '738', '739', '740', '741'
'744', '746', '751', '752', '756', '762', '764', '765', '766', '767'
'768', '769', '770', '771', '774', '775', '778', '779', '781', '782'
'783', '784', '785', '786', '788', '789', '790', '791', '792', '793'
'794', '795', '796', '797', '798', '799', '800', '801', '804', '805'
'806', '807', '809', '812', '813', '814', '815', '816', '820', '823'
'824', '825', '829', '831', '834', '836', '837', '838', '839', '840'
'841', '842', '843', '845', '846', '847', '848', '849', '850', '851'
'852', '853', '854', '855', '856', '857', '858']
# ------------------------------
# Comparison for Scraped Projects
# ------------------------------

# Convert lists to sets for set operations
set_styleC_scraped = set(styleC_scraped)
set_styleR_scraped = set(styleR_scraped)
set_empty = set(empty)

# Common projects in scraped lists (intersection)
common_scraped = set_styleC_scraped & set_styleR_scraped

# Projects unique to each style
unique_to_styleC_scraped = set_styleC_scraped - set_styleR_scraped
unique_to_styleR_scraped = set_styleR_scraped - set_styleC_scraped

# ------------------------------
# Comparison for Skipped Projects
# ------------------------------

set_styleC_skipped = set(styleC_skipped)
set_styleR_skipped = set(styleR_skipped)
set_empty = set(empty)

# Common projects in skipped lists (intersection)
common_skipped = set_styleC_skipped & set_styleR_skipped
common_skipped_missing = common_skipped & set_empty
skipped_but_not_missing = common_skipped - common_skipped_missing

# Projects unique to each style
unique_to_styleC_skipped = set_styleC_skipped - set_styleR_skipped
unique_to_styleR_skipped = set_styleR_skipped - set_styleC_skipped
unique_to_empty = set_empty - common_skipped

# ------------------------------
# Output the Results
# ------------------------------

print("=== Comparison for Scraped Projects ===")
print("Total Style C Scraped Projects:", len(styleC_scraped))
print("Total Style R Scraped Projects:", len(styleR_scraped))
print("Common Scraped Projects (count = {}):".format(len(common_scraped)))
print(sorted(common_scraped))
print("Projects unique to Style C (count = {}):".format(len(unique_to_styleC_scraped)))
print(sorted(unique_to_styleC_scraped))
print("Projects unique to Style R (count = {}):".format(len(unique_to_styleR_scraped)))
print(sorted(unique_to_styleR_scraped))

print("\n=== Comparison for Skipped Projects ===")
print("Total Style C Skipped Projects:", len(styleC_skipped))
print("Total Style R Skipped Projects:", len(styleR_skipped))
print("Common Skipped Projects (count = {}):".format(len(common_skipped)))
print(sorted(common_skipped))
print("Common Skipped Projects Missing (count = {}):".format(len(common_skipped_missing)))
print(sorted(common_skipped_missing))
print("Skipped Projects Not Missing (count = {}):".format(len(skipped_but_not_missing)))
print(sorted(skipped_but_not_missing))

print("Projects unique to Style C (count = {}):".format(len(unique_to_styleC_skipped)))
print(sorted(unique_to_styleC_skipped))
print("Projects unique to Style R (count = {}):".format(len(unique_to_styleR_skipped)))
print(sorted(unique_to_styleR_skipped))
print("Projects unique to Empty (count = {}):".format(len(unique_to_empty)))
print(sorted(unique_to_empty))

=== Comparison for Scraped Projects ===
Total Style C Scraped Projects: 87
Total Style R Scraped Projects: 30
Common Scraped Projects (count = 0):
[]
Projects unique to Style C (count = 87):
['495', '557', '559', '560', '568', '569', '576', '577', '581', '586', '588', '589', '593', '602', '606', '607', '632C', '643AB', '643AC', '643AE', '643AJ', '643D', '643E', '643F', '643G', '643J', '643O', '643R', '643S', '643X', '644', '645A', '650AC', '653B', '653E', '653EA', '653F', '654', '670', '671', '678', '679', '680', '687', '688', '695', '702', '705', '707', '708', '709', '714', '720', '723', '725', '738', '740', '744', '746', '751', '752', '762', '765', '768', '769', '775', '778', '779', '796', '797', '798', '799', '800', '805', '806', '809', '814', '815', '816', '820', '823', '824', '825', '829', '831', '855', '856']
Projects unique to Style R (count = 30):
['493', '510', '561', '565', '574', '583', '590', '608', '628', '643AM', '643AP', '643T', '644A', '649B', '649C', '650AA', '651A', '

In [23]:
# only 643A has a table 10, but was not scraped. other 17 are missing.

# Create Itemzied and Merged datasets

In [36]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/03_raw/rawdata_cluster1_4_style_C_originals.csv', dtype={'estimated_time_to_construct': str})

#df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


######################################################################################################################################
########################################
# STEP 0: CREATE DESCRIPTION COLUMN FROM COST ALLOCATION FACTOR


def move_non_numeric_text(value):
    """Move non-numeric, non-percentage text from cost allocation factor to description.
       If a value is moved, return None for cost allocation factor."""
    if isinstance(value, str):  # Ensure it's a string before processing
        if re.fullmatch(r"[\d,.]+%?", value):  # Check if it's numeric or a percentage
            return value  # Keep numeric or percentage values
        return None  # Clear the value if it's text (moved to description)
    return value  # Return as is for non-string values


def extract_non_numeric_text(value):
    """Extract non-numeric, non-percentage text from the cost allocation factor column."""
    if isinstance(value, str):  # Ensure it's a string before processing
        if re.fullmatch(r"[\d,.]+%?", value):  # Check if it's numeric or a percentage
            return None
        return value.strip()  # Return text entries as is
    return None  # Return None for non-string values



def clean_total_entries(value):
    """If the value starts with 'Total', remove numbers, commas, and percentage signs, keeping only 'Total'."""
    if isinstance(value, str) and value.startswith("Total"):
        return "Total"  # Keep only "Total"
    return value  # Leave other values unchanged









# Create the 'description' column from 'cost allocation factor'
#if 'cost allocation factor' in df.columns:
#    df['description'] = df['cost allocation factor'].apply(extract_non_numeric_text)
#    df['cost_allocation_factor'] = df['cost allocation factor'].apply(move_non_numeric_text)  # Clear moved values

######################################################################################################################################
########################################
#STEP 1 MERGE COLUMNS

def merge_columns(df):
    merge_columns_dict = {

        "upgrade": [
            "upgrade",
             
            ],

        "capacity": [
            "capacity",
            "MW",
            
        ],   



        "estimated_cost_x_1000": [
            

            "estimated cost x 1000",
            "estimated_cost",
            "estimated cost",
            
            "estimated cost x 1000 constant dollar_1",
            "estimated cost x 1000 constant dollar",
            "estimated cost x 1000 constant",
            "estimated cost x",
            "allocated_cost",
            "assigned cost",
            "allocated cost",
            "sum of allocated constant cost",
             
        ],    


        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            
            "escalated cost x 1000 constant dollar",
            "estimated cost x 1000 escalated",
            "allocated cost escalated",
            "estimated cost x 1000 escalated without itcca",
            "escalated cost x 1000",
            "sum of allocated escalated cost",
            "assigned cost escalated",
            
             

        ],

        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct",
            


        ],

        "total_estimated_cost_x_1000": [
            "total nu cost",
            "total cost constant"
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
       
         

        "description": ["description",  ],

        
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocation",
            "cost allocatio n factor",
            "cost allocati on factor",
            "project allocation",
            "percent allocation",

        ],
       
    }

    # Identify unnamed columns
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]
    if unnamed_columns:
        merge_columns_dict["description"].extend(unnamed_columns)

    for new_col, old_cols in merge_columns_dict.items():
        existing_cols = [col for col in old_cols if col in df.columns]
        if existing_cols:
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df

df = merge_columns(df)


######################################################################################################################################
########################################
# STEP 2: REMOVE DOLLAR SIGNED VALUES FROM 'estimated_time_to_construct'
######## Other clean up

def remove_dollar_values(value):
    """Remove dollar amounts (e.g., $3625.89, $3300) from 'estimated_time_to_construct'."""
    if isinstance(value, str) and re.search(r"^\$\d+(\.\d{1,2})?$", value.strip()):
        return None  # Replace with None if it's a dollar-signed number
    return value.strip() if isinstance(value, str) else value

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(remove_dollar_values)


## Remove ranodm number in Total row:    
# Apply cleaning function to "upgrade" column after merging
#if 'upgrade' in df.columns:
 #   df['upgrade'] = df['upgrade'].apply(clean_total_entries)


 
 

    
######################################################################################################################################
########################################
# STEP 3: DROP UNNEEDED COLUMNS

#df.drop(['unnamed_3', 'unnamed_15', 'unnamed_18', 'unnamed_16', 'estimated cost x 1000 escalated with itcca'], axis=1, inplace=True, errors='ignore')

df.drop(['cost', 'total cost','cost_1', 'interconnection facility element', '11 technicalrequirement', '11 technical requirement', 'unnamed_5','unnamed_2','unnamed_3',
         'preqc3', '4937 a', 'unnamed_6', 'unnamed_1'], axis=1, inplace=True)



######################################################################################################################################
########################################
#STEP 4: NAMING CONVENTION
def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]



# Convert estimated_time_to_construct to integer (remove decimals) and keep NaNs as empty
#df['estimated_time_to_construct'] = pd.to_numeric(df['estimated_time_to_construct'], errors='coerce').apply(lambda x: int(x) if pd.notna(x) else None)


 






######################################################################################################################################
########################################
#STEP 5: REMOVING TOTAL ROW, AS THE PDFS GIVE TOTAL NETWORK COST RATHER THAN BY RNU, LDNU AS WE HAD BEFORE
# Remove rows where upgrade is "Total" (case-insensitive)

df['tot'] = df.apply(
    lambda row: 'yes' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) 
        ) else 'no',
    axis=1
) 

# Now extract ONLY "Total" rows with a foolproof match
total_rows_df = df[df['tot'] == 'yes']

total_rows_df = total_rows_df.groupby(['q_id', 'type_of_upgrade', 'upgrade'], as_index=False).first()
total_rows_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/02_intermediate/costs_phase_1_cluster_1_4_style_C_total_network.csv', index=False) 
df = df[df['cost_allocation_factor'].str.strip().str.lower() != 'total']
 
df.drop('tot', axis=1, inplace= True)




######################################################################################################################################
########################################
# STEP 6: Move upgrade phrases like IRNU from upgrade column to a new column upgrade_classificatio and also replace type_of_upgrade with LDNU, CANU



# Define the list of phrases for upgrade classification
upgrade_phrases = ["IRNU", "GRNU", "CANU-D", "IRNU-A", "LDNU", "CANU-GR", "PNU", "CANU"]

 



#df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 14/03_raw/cluster_14_style_Q.csv', index=False)  


if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )


mappings = {
    "PTO’s Interconnection Facilities (Note 2)": "PTO_IF",
    "PTO’s Interconnectio n Facilities (Note 2)": "PTO_IF",
    "PTOs Interconnection Facilities": "PTO_IF",
    "PTOs Interconnectio n Facilities": "PTO_IF",
 "PTO": 'PTO_IF',
 "PNU": "OPNU",
 "Delivery Network Upgrades": "LDNU",
 "Delivery Network": "ADNU",
 "Plan of Service Reliability Network Upgrades": "RNU",
 "Distribution Upgrades": "LDNU",
 "PG&E Reliability Network Upgrades": "RNU",
 "SDG&E Delivery Network Upgrades": "LDNU",
 "SCE Delivery Upgrades": "LDNU",
 "SCE Distribution Upgrades": "LDNU",
 "SCE Reliability Network Upgrades for Short Circuit duty": "RNU",
 "SCE Network Upgrades": "RNU",
 "Plan of Service Distribution Upgrades": "LDNU",
 "PG&E Delivery Network Upgrades": "LDNU",
 "SCE Delivery Network Upgrades": "LDNU",

 "Reliability Network Upgrades": "RNU",
    "Local Delivery Network Upgrades": "LDNU",
    "Area Deliverability Upgrades": "ADNU",
    "Escalated Cost and Time to Construct for Interconnection Facilities, Reliability Network Upgrades, and Delivery Network Upgrades": "LDNU",
    "Distribution": "ADNU",
'Total PTO_IF': 'PTO_IF',
 'Total RNU': 'RNU',
 'Total LDNU': 'LDNU',
 'Total OPNU' : 'OPNU',
 'Total CANU': 'CANU',
 'Total LOPNU': 'LOPNU',
 'Total ADNU': 'ADNU',
}


def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type_of_upgrade",
        "upgrade",
        "description",
        "cost_allocation_factor",
        "estimated_cost_x_1000",
        "escalated_cost_x_1000",
        "total_estimated_cost_x_1000",
        "total_estimated_cost_x_1000_escalated",
        "estimated_time_to_construct",
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df     


if 'type_of_upgrade' in df.columns:
  
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )


if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()    

df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/03_raw/cluster_5_style_D.csv', index=False)

######################################################################################################################################
########################################
#STEP 7: Stable sort type of upgrade

def stable_sort_by_type_of_upgrade(df):
    """Performs a stable sort within each q_id to order type_of_upgrade while preserving row order in other columns."""
    
    # Define the custom sorting order for type_of_upgrade
    type_order = {"PTO_IF": 1, "RNU": 2, "LDNU": 3, "PNU": 4, "ADNU": 5}

    # Assign a numerical sorting key; use a high number if type_of_upgrade is missing
    df['sort_key'] = df['type_of_upgrade'].map(lambda x: type_order.get(x, 99))

    # Perform a stable sort by q_id first, then by type_of_upgrade using the custom order
    df = df.sort_values(by=['q_id', 'sort_key'], kind='stable').drop(columns=['sort_key'])

    return df

# Apply stable sorting
  


df = df.groupby(['q_id', 'type_of_upgrade', 'upgrade'], as_index=False).first()


df= reorder_columns(df)

#df = stable_sort_by_type_of_upgrade(df)  
#df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 14/03_raw/cluster_14_style_Q.csv', index=False)
######################################################################################################################################
########################################
# STEP 8: Remove $ signs and convert to numeric
 

def clean_currency(value):
    """
    Cleans a string by explicitly removing $, *, (Note 2), and similar patterns,
    then converts it to a numeric value.
    """
    if isinstance(value, str):
        # Explicitly remove $, *, and any "(Note ...)"
        value = value.replace('$', '').replace('*', '')
        value = re.sub(r'\(Note \d+\)', '', value)  # Remove patterns like "(Note 2)"
        value = value.replace(',', '').strip()  # Remove commas and extra spaces
    try:
        return pd.to_numeric(value)
    except ValueError:
        return pd.NA  # Return NaN for invalid entries
    




# Clean the specific columns
for col in ['estimated_cost_x_1000',]:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)






######################################################################################################################################
########################################
# STEP 9: Create Total rows


df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)


  

# Create Total rows
new_rows = []
columns_to_sum = ['estimated_cost_x_1000']
columns_to_populate = ['cluster', 'req_deliverability', 'latitude', 'longitude', 'capacity', 'point_of_interconnection']

for q_id, group in df.groupby('q_id', as_index=False):
    print(f"\nProcessing q_id: {q_id}")  # Debug print
    unique_upgrades = group['type_of_upgrade'].dropna().unique()
    for upgrade in unique_upgrades:
        if pd.isna(upgrade):
            continue
        
        rows = group[group['type_of_upgrade'] == upgrade]

        # Debug: Print current group
        print(f"\nChecking Upgrade: {upgrade}, Total Rows Present?:", 
              ( (group['item'] == 'no')).any())

        # Check if a Total row already exists for this (q_id, upgrade)
        total_exists = ((group['item'] == 'no')).any()
        
        if total_exists:
            print(f"Skipping Total row for {upgrade} (already exists).")
            continue
        
        total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings
        total_row['q_id'] = q_id
        total_row['type_of_upgrade'] = f"Total {upgrade}"
        total_row['item'] = 'no'

        # Populate specified columns from the existing row
        first_row = rows.iloc[0]
        for col in columns_to_populate:
            if col in df.columns:
                total_row[col] = first_row[col]

        # Sum the numeric columns
        for col in columns_to_sum:
            if col in rows.columns:
                total_row[col] = rows[col].sum()
            else:
                total_row[col] = 0  # Default to 0 if column is missing

        print(f"Creating Total row for {upgrade}")  # Debug print
        new_rows.append(total_row)

# Convert list to DataFrame and append
if new_rows:
    total_rows_df = pd.DataFrame(new_rows)
    print("\nNew Total Rows Created:\n", total_rows_df)  # Debug print
    df = pd.concat([df, total_rows_df], ignore_index=True)

df.reset_index(drop=True, inplace=True)


if 'type_of_upgrade' in df.columns:
  
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )


df = stable_sort_by_type_of_upgrade(df)

 


#: Move 'item' column next to 'type_of_upgrade'
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]




#  Remove "Total" values from cost_allocation_factor if they appear in type_of_upgrade
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if (
            pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])
        ) else row.get('cost_allocation_factor'),
        axis=1
    )
    

if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if 'Total' in str(row.get('cost_allocation_factor', '')) else row.get('cost_allocation_factor'),
        axis=1
    )



def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value


def clean_estimated_time(value):
    """
    Removes the word 'month' or 'months' (case insensitive) from the value.
    Leaves behind any numbers or number ranges (e.g. "6", "6-12").
    """
    if isinstance(value, str):
        # Remove 'month' or 'months' (case-insensitive), optionally with spaces around them.
        cleaned_value = re.sub(r'(?i)\s*months?\s*', '', value)
        return cleaned_value.strip()
    return value

# Then apply it to your column, for example with Pandas:
df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )

if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None
    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

numeric_columns = [
    'cost_allocation_factor',
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
    'total_estimated_cost_x_1000_escalated',
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
    'estimated_cost_x_1000_escalated_without_itcca',
    'adnu_cost_rate_x_1000_escalated'
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)
        df[col] = df[col].fillna(0)

if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    df = df.drop(columns=['original_order'])


   


#df= reorder_columns(df)


# Save itemized and totals separately
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
    itemized_df = itemized_df.drop_duplicates(subset=['q_id', 'type_of_upgrade', 'upgrade'])
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/02_intermediate/costs_phase_1_cluster_1_4_style_C_itemized.csv', index=False)

    totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
    totals_df = df[df['item'] == 'no'].drop(columns=totals_columns, errors='ignore')
    #totals_df = totals_df.drop_duplicates(subset=['q_id', 'type_of_upgrade', 'upgrade'])
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 3/02_intermediate/costs_phase_1_cluster_1_4_style_C_total.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_1_cluster_5_style_D_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_1_cluster_5_style_D_total.csv'.")



if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())


#df.to_csv('Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 14/03_raw/rawdata_cluster14_style_Q.csv')
#



Processing q_id: 495

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: RNU, Total Rows Present?: False
Creating Total row for RNU

Processing q_id: 557

Checking Upgrade: LDNU, Total Rows Present?: False
Creating Total row for LDNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: RNU, Total Rows Present?: False
Creating Total row for RNU

Processing q_id: 559

Checking Upgrade: LDNU, Total Rows Present?: False
Creating Total row for LDNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: RNU, Total Rows Present?: False
Creating Total row for RNU

Processing q_id: 560

Checking Upgrade: LDNU, Total Rows Present?: False
Creating Total row for LDNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: RNU, Total Rows Present?: False
Creating Total row for RNU

Processing q_id: 568

C